In [105]:
from utils.constants import OUTER_SEP, INNER_SEP
from classes.parser import Parser
from classes.action import Action
from classes.predicate import Predicate
import random

In [95]:
p = Parser('./problems/domain.txt', './problems/simple.txt', OUTER_SEP, INNER_SEP)

In [96]:
p.parse()

In [97]:
for i in p.init_state:
    print(i)

clear(a)
arm-empty()
on(a, b)
on-table(b)


In [98]:
for i in p.goals:
    print(i)

on-table(a)
clear(b)


In [99]:
print((p.ground_actions[0]))

PICK-UP
args:a
preconditions:clear(a),  on-table(a),  arm-empty()
add effects:holding(a)
delete effects:on-table(a),  clear(a),  arm-empty()


In [100]:
print(type(p.ground_actions[0])), print(type(p.init_state[0])), print(type(p.goals[0]))

<class 'classes.action.Action'>
<class 'classes.predicate.Predicate'>
<class 'classes.predicate.Predicate'>


(None, None, None)

In [101]:
p.ground_actions[0].args

['a']

In [102]:
p.ground_actions[0].preconditions[0].name

'clear'

# backward search

In [66]:
p.goals[0].name, p.goals[0].args

('on-table', ['a'])

In [84]:
def check_relevancy(act : Action, goal_list : list) -> bool:
    flag = False
    # as we do not have any negative precond or negative goal, so just we care about add effects
    for add_eff in act.add_effects:
        for g in goal_list:
            if add_eff == g:
                flag = True
                break
        if flag:
            break
    if not flag: 
        return False
    # check whether negative effects delete a goal
    for del_eff in act.delete_effects:
        for g in goal_list:
            if del_eff == g:
                return False
    return True

In [111]:
def gamma_inverse(act: Action, goals: list) -> list:
    for add_eff in act.add_effects:
        if add_eff in goals:
            goals.remove(add_eff)
    for precond in act.preconditions:
        goals.append(precond)
    return goals

In [112]:
def backward_search(init, goals, actions, trajectory):
    cur_goals = goals
    # check if init state satifies cur_goals
    flag = False
    for goal in cur_goals:
        for pred in init:
            if goal == pred : 
                flag = True
                break
        if flag :
            # check the next goal
            flag = False
        else : 
            break
    else : 
        print('problem sloved :))')
        return trajectory
    
    relevant_actions = []
    for act in actions:
        if check_relevancy(act, cur_goals):
            relevant_actions.append(act)
    while len(relevant_actions) > 0:
        # non-deterministically choose a relevant action
        act_index = random.randint(0,len(relevant_actions)-1)
        act = relevant_actions[act_index]
        relevant_actions.remove(act)
        new_goals = gamma_inverse(act, cur_goals.copy())
        
        # check new_goals is superset of old goals
        flag = False
        for old_goal in cur_goals:
            for new_goal in new_goals:
                if old_goal == new_goal:
                    flag = True
                    break
            if not flag :
                # new goals are superset of old goals and we struggled in a loop, so test the next relevant action
                break
            else : 
                flag = False
        # if loop did not detected
        else : 
            new_trajectory = trajectory.copy()
            new_trajectory.insert(0, act)
            backward_search(init, new_goals, actions, new_trajectory)
    
        

In [117]:
trajectory = []
t= backward_search(p.init_state, p.goals, p.ground_actions, trajectory)

In [119]:
p.init_state

In [120]:
p.goals

In [80]:
p.ground_actions[0].add_effects

In [83]:
cur_goals

In [81]:
cur_goals[0]

In [107]:
p.init_state

In [71]:
p.init_state[0].args

['a']